In [1]:
import pandas as pd
from sklearn.metrics import make_scorer, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from hyperopt import hp
import numpy as np
from hyperopt import Trials, tpe, fmin
import warnings
from imblearn.under_sampling import RandomUnderSampler
from joblib import load
import statistics
from sklearn.metrics import accuracy_score, f1_score, roc_auc_score, matthews_corrcoef, precision_score, recall_score
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv("output.csv",index_col=False)

In [3]:
rus = RandomUnderSampler(sampling_strategy='auto', random_state=42)

In [4]:
selected_columns=["locus tag","essential","DNA","protein sequence"]
Xs= df.drop(columns=selected_columns)
y=df["essential"]

In [5]:
from sklearn.model_selection import KFold

model_rf_lassoRFE = load('../../model/RF_model_FSS.joblib')
scores_fss_ac = []
scores_fss_mcc= []
scores_fss_f1 = []
scores_fss_auc = []
scores_fss_precision = []
scores_fss_recall = []
feature_rf = ['GC_Content', 'CAI', 'A', 'R', 'N', 'D', 'G', 'K', 'F', 'S', 'T', 'Y', 'V', 'nSE2', 'nSE3', 'nGE2', 'nGE3']
kf = KFold(n_splits=10, shuffle=True, random_state=42)

for train_index, test_index in kf.split(Xs):
    X_train_fold, X_test_fold = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train_undersampled, y_train_undersampled = rus.fit_resample(X_train_fold, y_train_fold)
    model_rf_lassoRFE.fit(X_train_undersampled[feature_rf], y_train_undersampled)
    y_predict = model_rf_lassoRFE.predict(X_test_fold[feature_rf])
    acc=accuracy_score(y_test, y_predict)
    scores_fss_ac.append(acc)
    mcc = matthews_corrcoef(y_test,y_predict)
    scores_fss_mcc.append(mcc)
    auc = roc_auc_score(y_test,y_predict)
    scores_fss_auc.append(auc)
    f1 = f1_score(y_test,y_predict)
    scores_fss_f1.append(f1)
    precision = precision_score(y_test,y_predict)
    scores_fss_precision.append(precision)
    recall = recall_score(y_test,y_predict)
    scores_fss_recall.append(recall)

In [6]:
print(scores_fss_ac)
print(statistics.mean(scores_fss_ac),statistics.variance(scores_fss_ac))

[0.5531400966183575, 0.6135265700483091, 0.6328502415458938, 0.5990338164251208, 0.5700483091787439, 0.5893719806763285, 0.5579710144927537, 0.48792270531400966, 0.5869565217391305, 0.5665859564164649]
0.5757407212455112 0.001577578809343551


In [7]:
model_xgb_nfs = load('../../model/XGB_model.joblib')
scores_nfs_ac = []
scores_nfs_mcc = []
scores_nfs_auc = []
scores_nfs_f1 = []
scores_nfs_precision = []
scores_nfs_recall = []
for train_index, test_index in kf.split(Xs):
    X_train_fold, X_test_fold = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train_undersampled, y_train_undersampled = rus.fit_resample(X_train_fold, y_train_fold)
    model_xgb_nfs.fit(X_train_undersampled, y_train_undersampled)
    y_predict = model_xgb_nfs.predict(X_test_fold)
    acc=accuracy_score(y_test, y_predict)
    scores_nfs_ac.append(acc)
    mcc = matthews_corrcoef(y_test,y_predict)
    scores_nfs_mcc.append(mcc)
    auc = roc_auc_score(y_test,y_predict)
    scores_nfs_auc.append(auc)
    f1 = f1_score(y_test,y_predict)
    scores_nfs_f1.append(f1)
    precision = precision_score(y_test,y_predict)
    scores_nfs_precision.append(precision)
    recall = recall_score(y_test,y_predict)
    scores_nfs_recall.append(recall)

In [8]:
print(scores_nfs_ac)
print(statistics.mean(scores_nfs_ac),statistics.variance(scores_nfs_ac))

[0.6545893719806763, 0.6714975845410628, 0.6618357487922706, 0.6183574879227053, 0.6183574879227053, 0.6884057971014492, 0.6280193236714976, 0.5869565217391305, 0.644927536231884, 0.6101694915254238]
0.6383116351428806 0.0009782718516277624


In [9]:
model_mlp_lasso = load('../../model/MLP_model_lasso.joblib')
scores_lasso_ac = []
scores_lasso_mcc = []
scores_lasso_auc = []
scores_lasso_f1 = []
scores_lasso_precision = []
scores_lasso_recall = []
feature_lasso = ['GC_Content', 'CAI', 'A', 'N', 'H', 'nSE3', 'nGE3']
for train_index, test_index in kf.split(Xs):
    X_train_fold, X_test_fold = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train_undersampled, y_train_undersampled = rus.fit_resample(X_train_fold, y_train_fold)
    model_mlp_lasso.fit(X_train_undersampled[feature_lasso], y_train_undersampled)
    y_predict = model_mlp_lasso.predict(X_test_fold[feature_lasso])
    acc=accuracy_score(y_test, y_predict)
    scores_lasso_ac.append(acc)
    mcc = matthews_corrcoef(y_test,y_predict)
    scores_lasso_mcc.append(mcc)
    auc = roc_auc_score(y_test,y_predict)
    scores_lasso_auc.append(auc)
    f1 = f1_score(y_test,y_predict)
    scores_lasso_f1.append(f1)
    precision = precision_score(y_test,y_predict)
    scores_lasso_precision.append(precision)
    recall = recall_score(y_test,y_predict)
    scores_lasso_recall.append(recall)

In [10]:
print(scores_lasso_ac)
print(np.mean(scores_lasso_ac),np.var(scores_lasso_ac))

[0.7101449275362319, 0.9178743961352657, 0.4057971014492754, 0.5289855072463768, 0.8719806763285024, 0.5144927536231884, 0.6256038647342995, 0.5096618357487923, 0.2463768115942029, 0.7481840193704601]
0.6079101893766595 0.03924175320708645


In [11]:
model_svm_rfe = load('../../model/SVM_model.joblib')
scores_rfe_ac = []
scores_rfe_mcc = []
scores_rfe_auc = []
scores_rfe_f1 = []
scores_rfe_precision = []
scores_rfe_recall = []
feature_rfe = ['CAI', 'R', 'N', 'D', 'C', 'Q', 'G', 'H', 'F', 'P', 'T', 'Y', 'nSE2', 'nGE2']
for train_index, test_index in kf.split(Xs):
    X_train_fold, X_test_fold = Xs.iloc[train_index], Xs.iloc[test_index]
    y_train_fold, y_test = y.iloc[train_index], y.iloc[test_index]
    X_train_undersampled, y_train_undersampled = rus.fit_resample(X_train_fold, y_train_fold)
    model_svm_rfe.fit(X_train_undersampled[feature_rfe], y_train_undersampled)
    y_predict = model_svm_rfe.predict(X_test_fold[feature_rfe])
    acc=accuracy_score(y_test, y_predict)
    scores_rfe_ac.append(acc)
    mcc = matthews_corrcoef(y_test,y_predict)
    scores_rfe_mcc.append(mcc)
    auc = roc_auc_score(y_test,y_predict)
    scores_rfe_auc.append(auc)
    f1 = f1_score(y_test,y_predict)
    scores_rfe_f1.append(f1)
    precision = precision_score(y_test,y_predict)
    scores_rfe_precision.append(precision)
    recall = recall_score(y_test,y_predict)
    scores_rfe_recall.append(recall)

In [12]:
print(scores_rfe_ac)
print(np.mean(scores_rfe_ac),np.var(scores_rfe_ac))

[0.6207729468599034, 0.7028985507246377, 0.5507246376811594, 0.5458937198067633, 0.533816425120773, 0.5652173913043478, 0.5241545893719807, 0.4855072463768116, 0.5845410628019324, 0.6053268765133172]
0.5718853446561626 0.0033189406249153667


In [13]:
import csv
def generateString(scores):
    mean_score = round(np.mean(scores),4)
    var_score = round(np.var(scores),4)
    return f'{mean_score}±{var_score}'
with open('resultEcoli.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Type','ACC','MCC','AUC','F1','Precision', 'Recall'])
    writer.writerow(['nFS',generateString(scores_nfs_ac),generateString(scores_nfs_mcc),generateString(scores_nfs_auc),generateString(scores_nfs_f1),generateString(scores_nfs_precision),generateString(scores_nfs_recall)])
    writer.writerow(['Lasso',generateString(scores_lasso_ac),generateString(scores_lasso_mcc),generateString(scores_lasso_auc),generateString(scores_lasso_f1),generateString(scores_lasso_precision),generateString(scores_lasso_recall)])
    writer.writerow(['RFE',generateString(scores_rfe_ac),generateString(scores_rfe_mcc),generateString(scores_rfe_auc),generateString(scores_rfe_f1),generateString(scores_rfe_precision),generateString(scores_rfe_recall)])
    writer.writerow(['FSS',generateString(scores_fss_ac),generateString(scores_fss_mcc),generateString(scores_fss_auc),generateString(scores_fss_f1),generateString(scores_fss_precision),generateString(scores_fss_recall)])